# Create Heatmaps for Weather Parameters

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Krasnaya Gorbatka,RU,2021-12-01 23:49:04,55.8703,41.7641,24.37,75,100,11.18
1,1,Atuona,PF,2021-12-01 23:49:04,-9.8000,-139.0333,78.58,79,57,21.27
2,2,Leh,IN,2021-12-01 23:49:04,34.1667,77.5833,18.97,50,97,3.02
3,3,Severo-Kurilsk,RU,2021-12-01 23:49:05,50.6789,156.1250,42.03,91,100,20.85
4,4,La Ronge,CA,2021-12-01 23:49:05,55.1001,-105.2842,14.25,85,90,6.91


One caveat to using gmaps: The data we use for any mapping must be either an integer or a floating-point decimal number. Let's check the data types for the columns of our DataFrame.

In [3]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

## Create a Maximum Temperature Heatmap


In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

Next, create the heatmap for the maximum temperature. The general syntax for creating a heatmap is as follows.

Google heatmaps do not plot negative numbers. If you have a maximum temperature that is less than 0 °F, then you will get an InvalidWeightException error for this line of code:

To remove the negative temperatures we can use a for loop to iterate through the max_temp and add the temperatures that are greater than 0 °F to a new list.



In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    # In the for loop, we're using the max() function to get the largest value between the temp and 0. 
    #If the temp is less than 0, then 0 will be added to the list in its place. Otherwise, 
    #the temp is added to the list
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature

# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]

# 2. Assign the weights variable to some values.
    # Google heatmaps do not plot negative numbers. If you have a maximum temperature that is less than 0 °F, then you will get an InvalidWeightException error for this line of code:
max_temp = city_data_df["Max Temp"]

# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure()

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
heat_layer = gmaps.heatmap_layer(locations, weights=temps)
                                 
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)

# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))